In [1]:
import os
#import cv2
import numpy as np
import pandas as pd
import model as m
from tqdm import tqdm
from keras.models import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.applications.xception import preprocess_input

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
model_image_size = (330, 440)
batch_size = 16

model = load_model('Xception3.h5')
gen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_generator = valid_gen.flow_from_directory('./imgs/valid3',  model_image_size, shuffle=False, batch_size=batch_size, class_mode="categorical")

In [ ]:
import matplotlib.pyplot as plt
import random
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

def show_heatmap_image(model_show, weights_show, image_file):
    status = ["safe driving",  " texting - right",  "phone - right",  "texting - left",  "phone - left",  
                  "operation radio", "drinking", "reaching behind", "hair and makeup", "talking"]
    plt.figure()
    img = cv2.imread(basedir+'valid/'+image_file)
    img = cv2.resize(img,  (model_image_size[1],model_image_size[0]))
    img_in = img.copy().astype(np.float32)
    img_in = preprocess_input(img_in)
    out, predictions = model_show.predict_on_batch(np.expand_dims(img_in, axis=0))
    predictions = predictions[0]
    out = out[0]
    
    pred_idx = np.argmax(predictions)
    pred = status[pred_idx]
    prediction = predictions[pred_idx]
    
    val_idx = int(image_file[image_file.rfind('/')-1])  
    val = status[val_idx]

    plt.title('Valid: %s | Pred: %s %.2f%%' % (val, pred, prediction*100))
    
    cam = prediction * np.matmul(out, weights_show)
    cam = cam[:,:,pred_idx]
    cam -= cam.min()
    cam /= cam.max()

    cam = cv2.resize(cam, (model_image_size[1],model_image_size[0]))
    heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
    heatmap[np.where(cam <= 0.2)] = 0

    out = cv2.addWeighted(img, 0.8, heatmap, 0.4, 0)

    plt.axis('off')
    plt.imshow(out[:,:,::-1])

In [ ]:
weights = model.layers[final_layer].get_weights()[0]
layer_output = model.layers[visual_layer].output
model2 = Model(model.input, [layer_output, model.output])